In [96]:
import pandas as pd
import os
import configparser
from typing import List

In [17]:
def getBandNames(cat,bandIDs):
    return [list(cat.columns)[c-1] for c in bandIDs]

In [118]:
dirCat=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\catalogs\forPaper\v05'
dirConfigs=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR'
dirExp=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\experimentsArticle\Vis_UV_IR'
fileNameCat=r'05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors.csv'
idcolumn='Seq'
zspeccolumn='z_spec'
cat=pd.read_csv(os.path.join(dirCat,fileNameCat))

In [92]:
statsTest=pd.DataFrame(columns=['Exp','Bands','N bands','meanNorm','sigmaNorm','nmadNorm','outliers15Norm'])
for expDir in os.listdir(dirExp):
    rec={'Exp':expDir}
    conf=next(f for f in os.listdir(dirConfigs) if f.startswith(expDir))
    config = configparser.ConfigParser()
    config.read(os.path.join(dirConfigs,conf))
    bandId=config['Test Input Info']['photoColumnIdxList']
    rec['Bands']=conf.split('colors_')[1].replace('.ini','').replace('_',', ')
    rec['N bands']=len([int(d) for d in bandId.split(',')])
    for f in os.listdir(os.path.join(dirExp,expDir)):
        if f.startswith('METAPHOR_teststats'):
            df=pd.read_csv(os.path.join(dirExp,expDir,f),index_col='Indicator')
            rec['meanNorm']=df.loc['meanNorm'][0]
            rec['sigmaNorm']=df.loc['sigmaNorm'][0]
            rec['nmadNorm']=df.loc['nmadNorm'][0]
            rec['outliers15Norm']=df.loc['outliers15Norm'][0]
            statsTest=statsTest.append(rec,ignore_index=True)

In [93]:
statsTest

,Exp,Bands,N bands,meanNorm,sigmaNorm,nmadNorm,outliers15Norm
0,exp001,Vis,5,-0.003168,0.055026,0.023652,2.242308
1,exp003,IR,4,-0.010990,0.108837,0.065733,13.488615
2,exp004,"Vis, UV",6,-0.003315,0.051840,0.019833,1.929428
3,exp005,"Vis, IR",9,-0.003430,0.053220,0.020192,2.259691
4,exp006,"Vis, UV, IR",10,-0.001763,0.052291,0.018334,2.068486
5,exp007,"Vis, UV, Ymagap3",7,-0.002234,0.049078,0.019289,2.103250
6,exp008,"Vis, UV, Jmagap3",7,-0.003833,0.052721,0.018382,2.085868
7,exp009,"Vis, UV, Hmagap3",7,-0.002686,0.046731,0.020011,1.755606
8,exp010,"Vis, UV, Ksmagap3",7,-0.001554,0.049794,0.019903,1.964193
9,exp011,"Vis, UV, IB574ap3",7,-0.002781,0.048786,0.018996,1.772988


In [98]:
(statsTest.columns==['Exp','Bands','N bands','meanNorm','sigmaNorm','nmadNorm','outliers15Norm'])

True

In [155]:
def addResults(statsTab, dirExp:str, dirConfigs:str,
               expNums:List[int],case:str='test',splitConfNameBy:str='colors_'):
    if not all(statsTab.columns==['Exp','N obj','Bands','N bands','meanNorm','sigmaNorm','nmadNorm','outliers15Norm']):
        print('statsTab columns do not match the requirements')
        return
    case='METAPHOR_'+case+'stats'
    for exp in expNums:
        expDir='exp'+str(exp).zfill(3)
        rec={'Exp':expDir}
        try:
            conf=next(f for f in os.listdir(dirConfigs) if f.startswith(expDir))
            config = configparser.ConfigParser()
            config.read(os.path.join(dirConfigs,conf))
            bandId=config['Test Input Info']['photoColumnIdxList']
            rec['Bands']=conf.split(splitConfNameBy)[1].replace('.ini','').replace('_',', ')
            rec['N bands']=len([int(d) for d in bandId.split(',')])
        except:
            print('No config '+expDir+' found')
            rec['Bands']=''
            rec['N bands']=0
        try:
            for f in os.listdir(os.path.join(dirExp,expDir)):
                if f.startswith(case):
                    df=pd.read_csv(os.path.join(dirExp,expDir,f),index_col='Indicator')
                    rec['meanNorm']=df.loc['meanNorm'][0]
                    rec['sigmaNorm']=df.loc['sigmaNorm'][0]
                    rec['nmadNorm']=df.loc['nmadNorm'][0]
                    rec['outliers15Norm']=df.loc['outliers15Norm'][0]
                    rec['N obj']=df.loc['EstimatesInRange'][0]
                    statsTab=statsTab.append(rec,ignore_index=True)
        except:
            print('No directory '+expDir+' found')
    return statsTab

In [161]:
statsTest=pd.DataFrame(columns=['Exp','N obj','Bands','N bands','meanNorm','sigmaNorm','nmadNorm','outliers15Norm'])

In [162]:
statsTest=addResults(statsTest, dirExp=dirExp, dirConfigs=dirConfigs, expNums=list(range(1,27)))

No directory exp002 found


In [163]:
statsTest=addResults(statsTest, dirExp=dirExp.replace('Vis_UV_IR','DirtySample'), 
           dirConfigs=dirConfigs.replace('Vis_UV_IR','dirtySample'), 
           splitConfNameBy='1p2_',expNums=list(range(27,29)))

In [164]:
statsTest

,Exp,N obj,Bands,N bands,meanNorm,sigmaNorm,nmadNorm,outliers15Norm
0,exp001,5753.0,Vis,5,-0.003168,0.055026,0.023652,2.242308
1,exp003,5753.0,IR,4,-0.010990,0.108837,0.065733,13.488615
2,exp004,5753.0,"Vis, UV",6,-0.003315,0.051840,0.019833,1.929428
3,exp005,5753.0,"Vis, IR",9,-0.003430,0.053220,0.020192,2.259691
4,exp006,5753.0,"Vis, UV, IR",10,-0.001763,0.052291,0.018334,2.068486
5,exp007,5753.0,"Vis, UV, Ymagap3",7,-0.002234,0.049078,0.019289,2.103250
6,exp008,5753.0,"Vis, UV, Jmagap3",7,-0.003833,0.052721,0.018382,2.085868
7,exp009,5753.0,"Vis, UV, Hmagap3",7,-0.002686,0.046731,0.020011,1.755606
8,exp010,5753.0,"Vis, UV, Ksmagap3",7,-0.001554,0.049794,0.019903,1.964193
9,exp011,5753.0,"Vis, UV, IB574ap3",7,-0.002781,0.048786,0.018996,1.772988


In [165]:
statsTest=addResults(statsTest, dirExp=dirExp.replace('Vis_UV_IR','FeatureSelection'), 
           dirConfigs=dirConfigs.replace('Vis_UV_IR','FeatureSelection'), 
           splitConfNameBy='1p2_',expNums=list(range(29,31)))

In [168]:
statsTest.round(3).to_latex(index=False)

'\\begin{tabular}{lrllrrrr}\n\\toprule\n    Exp &   N obj &                                Bands & N bands &  meanNorm &  sigmaNorm &  nmadNorm &  outliers15Norm \\\\\n\\midrule\n exp001 &  5753.0 &                                  Vis &       5 &    -0.003 &      0.055 &     0.024 &           2.242 \\\\\n exp003 &  5753.0 &                                   IR &       4 &    -0.011 &      0.109 &     0.066 &          13.489 \\\\\n exp004 &  5753.0 &                              Vis, UV &       6 &    -0.003 &      0.052 &     0.020 &           1.929 \\\\\n exp005 &  5753.0 &                              Vis, IR &       9 &    -0.003 &      0.053 &     0.020 &           2.260 \\\\\n exp006 &  5753.0 &                          Vis, UV, IR &      10 &    -0.002 &      0.052 &     0.018 &           2.068 \\\\\n exp007 &  5753.0 &                     Vis, UV, Ymagap3 &       7 &    -0.002 &      0.049 &     0.019 &           2.103 \\\\\n exp008 &  5753.0 &                     Vis, UV, Jmag

In [78]:
statsTrain=pd.DataFrame(columns=['Exp','Bands','N bands','meanNorm','sigmaNorm','nmadNorm','outliers15Norm'])
for expDir in os.listdir(dirExp):
    rec={'Exp':expDir}
    conf=next(f for f in os.listdir(dirConfigs) if f.startswith(expDir))
    config = configparser.ConfigParser()
    config.read(os.path.join(dirConfigs,conf))
    bandId=config['Test Input Info']['photoColumnIdxList']
    rec['Bands']=conf.split('colors_')[1].replace('.ini','').replace('_',', ')
    rec['N bands']=len([int(d) for d in bandId.split(',')])
    for f in os.listdir(os.path.join(dirExp,expDir)):
        if f.startswith('METAPHOR_trainstats'):
            df=pd.read_csv(os.path.join(dirExp,expDir,f),index_col='Indicator')
            rec['meanNorm']=df.loc['meanNorm'][0]
            rec['sigmaNorm']=df.loc['sigmaNorm'][0]
            rec['nmadNorm']=df.loc['nmadNorm'][0]
            rec['outliers15Norm']=df.loc['outliers15Norm'][0]
            statsTrain=statsTrain.append(rec,ignore_index=True)

In [79]:
statsTrain

,Exp,Bands,N bands,meanNorm,sigmaNorm,nmadNorm,outliers15Norm
0,exp001,Vis,5,-0.002636,0.053453,0.023821,2.197720
1,exp003,IR,4,-0.010592,0.106553,0.065893,13.275721
2,exp004,"Vis, UV",6,-0.002132,0.048137,0.019778,1.780526
3,exp005,"Vis, IR",9,-0.002056,0.048866,0.020322,1.877375
4,exp006,"Vis, UV, IR",10,-0.001572,0.044910,0.018023,1.653878
5,exp007,"Vis, UV, Ymagap3",7,-0.002170,0.048228,0.019355,1.758176
6,exp008,"Vis, UV, Jmagap3",7,-0.001886,0.046715,0.018775,1.706027
7,exp009,"Vis, UV, Hmagap3",7,-0.002324,0.048295,0.020076,1.817775
8,exp010,"Vis, UV, Ksmagap3",7,-0.002019,0.047372,0.019212,1.780526
9,exp011,"Vis, UV, IB574ap3",7,-0.002286,0.048491,0.018907,1.817775


In [171]:
statsTest=addResults(statsTest, dirExp=dirExp, 
           dirConfigs=dirConfigs.replace('Vis_UV_IR','Narrows'), 
           splitConfNameBy='colors_',expNums=list(range(31,45)))

,Exp,N obj,Bands,N bands,meanNorm,sigmaNorm,nmadNorm,outliers15Norm
29,exp031,5753.0,"Vis, UV, J, IA484",8,-0.001616,0.051252,0.018409,2.085868
30,exp032,5753.0,"Vis, UV, J, IA527",8,-0.002804,0.052532,0.019352,1.929428
31,exp033,5753.0,"Vis, UV, J, IA624",8,-0.003242,0.050418,0.019361,1.720841
32,exp034,5753.0,"Vis, UV, J, IA679",8,-0.001274,0.050068,0.020488,1.790370
33,exp035,5753.0,"Vis, UV, J, IA738",8,-0.002112,0.047842,0.018783,1.964193
34,exp036,5753.0,"Vis, UV, J, IA767",8,-0.003180,0.050958,0.019441,1.946810
35,exp037,5753.0,"Vis, UV, J, IB427",8,-0.001219,0.046426,0.019554,1.842517
36,exp038,5753.0,"Vis, UV, J, IB464",8,-0.003290,0.053314,0.019302,1.912046
37,exp039,5753.0,"Vis, UV, J, IB505",8,-0.002843,0.053052,0.018417,1.859899
38,exp040,5753.0,"Vis, UV, J, IB574",8,-0.001726,0.047804,0.019472,1.894664


In [172]:
statsTest.iloc[29:].round(3).to_latex(index=False)

'\\begin{tabular}{lrllrrrr}\n\\toprule\n    Exp &   N obj &              Bands & N bands &  meanNorm &  sigmaNorm &  nmadNorm &  outliers15Norm \\\\\n\\midrule\n exp031 &  5753.0 &  Vis, UV, J, IA484 &       8 &    -0.002 &      0.051 &     0.018 &           2.086 \\\\\n exp032 &  5753.0 &  Vis, UV, J, IA527 &       8 &    -0.003 &      0.053 &     0.019 &           1.929 \\\\\n exp033 &  5753.0 &  Vis, UV, J, IA624 &       8 &    -0.003 &      0.050 &     0.019 &           1.721 \\\\\n exp034 &  5753.0 &  Vis, UV, J, IA679 &       8 &    -0.001 &      0.050 &     0.020 &           1.790 \\\\\n exp035 &  5753.0 &  Vis, UV, J, IA738 &       8 &    -0.002 &      0.048 &     0.019 &           1.964 \\\\\n exp036 &  5753.0 &  Vis, UV, J, IA767 &       8 &    -0.003 &      0.051 &     0.019 &           1.947 \\\\\n exp037 &  5753.0 &  Vis, UV, J, IB427 &       8 &    -0.001 &      0.046 &     0.020 &           1.843 \\\\\n exp038 &  5753.0 &  Vis, UV, J, IB464 &       8 &    -0.003 &      0